# Imports

In [ ]:
import os
import time
import random
import numpy as np

import tensorflow as tf
import tensorflow.contrib.layers as layers

import gym
from gym import spaces
from PIL import Image
from collections import deque, namedtuple
from IPython.display import HTML
from IPython import display
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
from environments import Snake
from visual_utils import AgentViz
from methods import DistQNetwork, ReplayMemory
from agents import SnakeDistDQNAgent

# NN training

In [ ]:
aa = SnakeDistDQNAgent(model_name="c51_baseline", num_atoms=51)

In [ ]:
aa.set_parameters(max_episode_length=1000, replay_memory_size=50000, replay_start_size=50, discount_factor=1)

In [ ]:
aa.train(gpu_id=2, from_epoch=0)

# Visualization

In [ ]:
aa.play(gpu_id=2, max_episode_length=100, from_epoch=234)